In [21]:
from query_handler import QueryHandler
from sql_client import SQLClient

import json
from collections import namedtuple

# Aghanim's Labyrinth Custom Abilities 

In [7]:
ability_constants_handler = QueryHandler(f'./queries/constants_custom_ability.txt')
r = ability_constants_handler.make_query()

query_result = json.loads(r)
abilities = query_result['data']['constants']['customAbilities']

In [15]:
unnested_abilities = []
for ability in abilities:
    try:
        ability['displayName'] = ability['language']['displayName']
        ability['description'] = ability['language']['description']
        del ability['language']
    except:
        ability['displayName'] = None
        ability['description'] = None
    unnested_abilities.append(ability)

In [28]:
sqlclient = SQLClient()
cur = sqlclient.cur

for ability in unnested_abilities:
    try:
        cur.execute("""INSERT INTO customAbilityConstants (id, name, abilityName, displayName, description) VALUES (%s, %s, %s, %s, %s)""",
        (ability['id'], ability['name'], ability['abilityName'], ability['displayName'], ability['description']))
    except:
        print(f'Error inserting {ability["id"]}')

sqlclient.conn.commit()
sqlclient.close_connection()

# Heroes / Abilities (Normal Dota)

In [9]:
hero_constants_handler = QueryHandler(f'./queries/constants_hero.txt')
r = hero_constants_handler.make_query()

query_result = json.loads(r)
heroes = query_result['data']['constants']['heroes']

Schema:
```
CREATE TABLE const_heroes(
    id SMALLINT,
    name TEXT,
    displayName TEXT,
    shortName TEXT,
    aliases TEXT[],
    PRIMARY KEY(id)
)

CREATE TABLE const_abilities(
    abilityId SMALLINT,
    heroId SMALLINT,
    slot SMALLINT,
    name TEXT,
    displayName TEXT,
    PRIMARY KEY(abilityId),
    FOREIGN KEY(heroId) REFERENCES const_heroes(id)
);
```

In [16]:
Hero = namedtuple('hero', ['id', 'name', 'displayName', 'shortName', 'aliases'])
Ability = namedtuple('ability', ['abilityId', 'heroId', 'slot', 'name', 'displayName'])

In [18]:
heroes_to_insert = []
abilities_to_insert = []

for hero in heroes:
    hero_tup = Hero(hero['id'], hero['name'], hero['displayName'], hero['shortName'], hero['aliases'])
    for ability in hero['abilities']:
        ability_tup = Ability(ability['abilityId'], hero['id'], ability['slot'], ability['ability']['name'], ability['ability']['language']['displayName'])
        abilities_to_insert.append(ability_tup)
    heroes_to_insert.append(hero_tup)

In [27]:
heroes[0]['aliases']

['am']

In [31]:
b''.join(list((cur.mogrify("(%s,%s,%s,%s,%s)", hero) for hero in heroes_to_insert)))

b"(1,'npc_dota_hero_antimage','Anti-Mage','antimage',ARRAY['am'])(2,'npc_dota_hero_axe','Axe','axe','{}')(3,'npc_dota_hero_bane','Bane','bane','{}')(4,'npc_dota_hero_bloodseeker','Bloodseeker','bloodseeker',ARRAY['bs'])(5,'npc_dota_hero_crystal_maiden','Crystal Maiden','crystal_maiden',ARRAY['cm','rylai'])(6,'npc_dota_hero_drow_ranger','Drow Ranger','drow_ranger','{}')(7,'npc_dota_hero_earthshaker','Earthshaker','earthshaker','{}')(8,'npc_dota_hero_juggernaut','Juggernaut','juggernaut','{}')(9,'npc_dota_hero_mirana','Mirana','mirana','{}')(10,'npc_dota_hero_morphling','Morphling','morphling','{}')(11,'npc_dota_hero_nevermore','Shadow Fiend','nevermore',ARRAY['sf'])(12,'npc_dota_hero_phantom_lancer','Phantom Lancer','phantom_lancer',ARRAY['pl'])(13,'npc_dota_hero_puck','Puck','puck','{}')(14,'npc_dota_hero_pudge','Pudge','pudge','{}')(15,'npc_dota_hero_razor','Razor','razor','{}')(16,'npc_dota_hero_sand_king','Sand King','sand_king',ARRAY['sk'])(17,'npc_dota_hero_storm_spirit','Storm Sp

In [36]:
sqlclient = SQLClient()
cur = sqlclient.cur

hero_rows = b','.join(cur.mogrify("(%s,%s,%s,%s,%s)", hero) for hero in heroes_to_insert)
cur.execute(b"INSERT INTO const_heroes VALUES " + hero_rows)

ability_rows = b','.join(cur.mogrify("(%s,%s,%s,%s,%s)", ability) for ability in abilities_to_insert)
cur.execute(b"INSERT INTO const_abilities VALUES " + ability_rows)

sqlclient.conn.commit()
sqlclient.close_connection()